#Lab.12 / IBM3202 – Modelamiento estructural *ab initio* usando Rosetta

#Aspectos teóricos.
¿Por qué podemos realizar predicción de estructuras para una secuencia dada?

Como ya vimos en el tutorial de modelado por homología, hay una creciente necesidad de resolver la estructura tridimensional de las proteínas. Dado que la evolución ha avanzado a partir de un número finito de familias de proteínas, generalmente hay proteínas estrechamente relacionadas que ya han sido resueltas y pueden utilizarse en estrategias como el mencionado modelado por homología. Sin embargo, también existe la posibilidad de encontrarse con una proteína sin homólogos cercanos estructuralmente resueltos. En este último caso, podemos recurrir a la modelización derivada de primeros principios, comúnmente conocida como modelado *Ab initio*, que es independiente de plantillas.


Como hemos discutido durante el curso, hay muchas formas de predecir la estructura de una proteína solo conociendo su secuencia: modelado comparativo basado en homología, determinación de acoplamientos coevolutivos utilizando una miríada de secuencias de proteínas homólogas y predicción de estructuras ab initio. *Ab initio* proviene del latín y significa "desde primeros principios", en el cual la idea es llegar a la estructura de una proteína solo conociendo los principios que rigen el plegamiento de proteínas.


¿De dónde provienen estos principios? Como hemos visto en nuestras conferencias, el creciente número de estructuras de proteínas resueltas por diferentes métodos (rayos X, RMN, criomicroscopía electrónica) ha permitido definir que, independientemente de su secuencia primaria, las proteínas no exploran todas las configuraciones posibles. Esta observación tan simple ha permitido un uso adicional de esta información para realizar análisis estadísticos de las diferentes conformaciones que suelen explorar las secuencias de proteínas: entornos típicos de aminoácidos y distancias entre aminoácidos en cualquier proteína dada, predicciones de estructuras secundarias basadas solo en información de secuencias, geometrías de ángulos y distancias entre elementos de estructura secundaria, distribuciones dihedrales de espinas dorsales y cadenas laterales, etc. Luego, esta información se puede utilizar para definir funciones de energía que nos permiten determinar qué estructuras son más probables para una secuencia dada (un problema inverso al plegamiento).


Si bien este método se ha utilizado para varias predicciones de estructuras de proteínas, el crecimiento exponencial del espacio conformacional de una proteína que puede explorar una secuencia de proteína dada con respecto a su longitud de proteína limita severamente su uso a proteínas con menos de 100 residuos. La lógica detrás de este número exacto es que un dominio de proteína, es decir, considerado como una región (secuencia) de una proteína que puede plegarse independientemente en una estructura dada por sí misma, típicamente tiene un máximo en su distribución de longitud típica en ~100 residuos para la mayoría de las proteínas (**Figura 1**). Por lo tanto, parece haber un número óptimo de residuos que permite a las proteínas formar un núcleo hidrofóbico lo suficientemente grande como para plegarse de manera estable.


<figure>
<center>
<img src="https://ars.els-cdn.com/content/image/1-s2.0-S1359027898000042-gr3.gif"/>

<figcaption>FIGURA 1. Distribución de longitudes de dominios de proteínas. La flecha representa el máximo de la distribución, que a su vez representa el mínimo de una función de energía de plegamiento que depende únicamente del costo entrópico de empaquetar un n-poliémero, el costo de hidratación para su radio de giro y la entalpía interna promedio de la formación de contactos.  <br> Adapted from Xu & Nussinov (1998)<i> Folding & Design.</i></figcaption></center>
</figure>

Aunque esto se usa como una regla general, hay muchas excepciones, como se muestra en la distribución anterior, de dominios que se pliegan bien por debajo o más allá de la longitud óptima de 100 ± 40 residuos. Algunas proteínas extremadamente cortas pueden adoptar estructuras estables en solución (por ejemplo, la horquilla β estable que estudiamos a través de la dinámica molecular, o la cabeza de la villina que estamos a punto de plegar), y proteínas inmensamente grandes que pueden formar un núcleo hidrofóbico enorme. Esta distribución ruidosa destaca que no hay una solución única para el problema del plegamiento de proteínas, y los principios que rigen el plegamiento de proteínas globulares ideales y de tamaño regular pueden no ser los mismos que los de dominios pequeños atípicos. Hoy pondremos eso a prueba.


#Resumen general

En este tutorial, utilizaremos Rosetta para predecir la estructura de la proteína de 35 residuos de la cabeza de la villina de G. gallus. Aunque compilar puede ser complicado y la documentación está dispersa y oscura a través de decenas de versiones diferentes y pequeñas modificaciones no anunciadas, Rosetta es, con mucho, el motor más exitoso en el diseño de novo y la predicción de estructuras de proteínas (Kaufmann *et al.* (2010) *Biochemistry*).

En resumen, realizaremos las siguientes tareas:

- Parte I – Preparar las bibliotecas de fragmentos necesarias para la predicción de la estructura de proteínas con Rosetta**

- Parte II – Ejecutar Rosetta para generar 10 modelos ab initio**

- Parte III – Analizar los resultados utilizando py3Dmol y Rosetta**

- Parte IV – Preparar un mayor número de modelos (>10,000) utilizando dos bibliotecas de fragmentos diferentes**


##Resumen del protocolo de plegamiento


Según se indica en la [documentación de AbinitioRelax](https://):

> La aplicación AbinitioRelax consta de dos pasos principales.
1. El primer paso es una búsqueda basada en fragmentos y de baja resolución a través del espacio conformacional utilizando una función de puntuación "centroide" basada en el conocimiento que favorece las características similares a las proteínas (Abinitio).
2. El segundo paso opcional es el refinamiento a nivel de átomos utilizando el campo de fuerza completo de Rosetta (Relax). El paso "Relax" es considerablemente más intensivo en cálculos y lleva más tiempo que el primer paso.

También se comenta el hecho de que:

> Para aumentar el muestreo conformacional, esta aplicación se puede paralelizar fácilmente ejecutando numerosos trabajos, cada uno utilizando una semilla de número aleatorio única (consultar la sección de opciones a continuación). Esto se hace típicamente mediante la presentación de múltiples trabajos a un clúster de computadoras o una cuadrícula distribuida. Dado que la función de energía a nivel de átomos es muy sensible a las interacciones atómicas imperfectas y habrá más ruido con un muestreo insuficiente, la convergencia hacia la estructura nativa puede requerir una cantidad significativa de muestreo. Además, para aumentar la probabilidad de muestrear la topología correcta, también se pueden ejecutar conjuntos diversos de secuencias homólogas, preferiblemente con cambios en la secuencia que puedan tener un mayor impacto en el muestreo, como eliminaciones y diferencias en posiciones conservadas, ya que un homólogo puede converger hacia la estructura nativa con significativamente menos muestreo (consultar Bradley *et al.* (2005). Toward high-resolution de novo structure prediction for small proteins. *Science*, 309(5742), 1868-1871).


## Folding.py
Este script realiza la inserción de fragmentos para una secuencia de proteínas de entrada. La secuencia puede ser explícita, en un archivo FASTA o en un archivo PDB. También se deben proporcionar dos archivos de fragmentos, preferiblemente uno más largo que el otro. La inserción de fragmentos está acompañada por una evaluación de Monte Carlo que permite que la conformación escape de mínimos locales. Las estructuras de salida de este protocolo están destinadas a pasar a un paso de refinamiento (como el de refinement.py) para producir estimaciones razonables de la conformación de la proteína.

Instrucciones:

1. Asegúrese de que su archivo PDB esté en el directorio actual.
2. Ejecute el script:
   - Desde la línea de comandos:
      ```
      python D060_Folding.py
      ```
   - Desde python/ipython:
      ```python
      run D060_Folding.py
      ```

Autor: Evan H. Baugh
Revisado y motivado por Robert Schleif

Actualizado por Boon Uranukul, 9/6/12
Inicialización simplificada de semillas constantes especiales ~ Labonte


Referencias:
    P. Bradley, K. Misura, and D. Baker, "Toward high-resolution de novo
        structure prediction for small proteins", *Science* 309 (5742)
        1868-1871 (2005).



## En el protocolo completo de plegamiento, pasos incluídos en la definición de "sample_folding"



Este script de ejemplo está configurado para su uso con:
  - argumentos de línea de comandos,
  - ejecución predeterminada dentro de un intérprete de Python,
  - o para importarse dentro de un intérprete de Python, exponiendo el método sample_folding.

El método sample_folding realiza los siguientes pasos:
1. Crea una pose a partir de la secuencia deseada (átomos completos).
2. Establece todos los ángulos de torsión del esqueleto de la pose.
3. Crea una copia de referencia (átomos completos) de la pose.
4. Crea Movers para cambiar entre átomos completos y centroide.
5. Convierte ambas poses a centroide.
6. Crea un MoveMap con todos los ángulos de torsión del esqueleto libres.
7. Configura ClassicFragmentMovers para insertar fragmentos de estructuras de esqueleto para fragmentos largos y cortos.
8. Crea RepeatMovers para los fragmentos largos y cortos.
9. Crea un PyMOL_Observer para ver la salida intermedia.
10. Crea funciones de puntuación de baja y alta resolución.
11. Configura RepeatMover en un TrialMover de un SequenceMover establece el TrialMover

    - Crea un SequenceMover con:
        - RepeatMover en el Mover de fragmento largo.
        - RepeatMover en el Mover de fragmento corto.

    - Crea un objeto MonteCarlo para evaluar movimientos.
    - Crea el TrialMover (en el SequenceMover).
    - Crea el RepeatMover (en el TrialMover).
12. Crea un (Py)JobDistributor para administrar múltiples trayectorias.
13. Almacena la evaluación de la puntuación original (centroide).
14. Realiza el plegamiento de baja resolución:
    - Configura variables necesarias
        - Recarga la pose inicial (centroide).
        - Cambia el PDBInfo.name de la pose, para el PyMOLMover.
        - Restablece el objeto MonteCarlo (para el TrialMover).
    - Realiza el muestreo y la evaluación utilizando el último RepeatMover.
    - Convierte el decoy de puntuación más baja a átomos completos utilizando el SequenceMover.
    - Exporta la estructura del decoy (puntuación más baja).
        - Recupera el decoy de puntuación más baja (centroide).
        - Almacena la puntuación del decoy.
        - Convierte el decoy a átomos completos.
        - Adivina posibles puentes disulfuro.
        - Salida de la estructura del decoy utilizando el PyJobDistributor.
        - Exporta la estructura del decoy utilizando el PyMOL_Observer.
15. Muestra las evaluaciones de la puntuación.

El segundo método, guess_disulfides, se llama en sample_folding para imprimir residuos de cisteína que están cerca uno del otro en la estructura del decoy.


# Parte I: Preparar las bibliotecas de fragmentos necesarias para la predicción de la estructura de proteínas usando Rosetta


La secuencia de la proteína para la cual queremos predecir su estructura, el mutante **G34L** del subdominio de la cabeza de la proteína **Villin** de *G. gallus*, es la siguiente:
> MLSDEDFKAFGMTRSAFANLPLWKQQNLKKEKLLF

Lo primero que necesitamos hacer es generar una biblioteca de fragmentos que necesitaremos para modelar nuestra proteína. Como recordarás de las conferencias, estas bibliotecas son esenciales para cerrar bucles entre segmentos y determinar la estructura local más probable para una secuencia dada.

Para generar estas bibliotecas, recomendamos utilizar **Robetta (http://old.robetta.org/)**, un servidor web que está a cargo del mismo laboratorio encargado de mantener Rosetta. Aquí, generalmente se irá a la sección "Submit" del servicio de la Biblioteca de Fragmentos, y luego se ingresará el nombre de la proteína objetivo en el campo "Target Name" y su secuencia completa en formato Fasta en el campo "Paste Fasta". Finalmente, normalmente se presionaría "Submit" para iniciar la generación de la biblioteca de fragmentos.


Dependiendo de la longitud de la secuencia de la proteína, la generación de estas bibliotecas de fragmentos puede tardar varias horas. Por lo tanto, estamos proporcionando estos archivos para su trabajo ininterrumpido (0- fragment_libraries/villin folder). Si echas un vistazo rápido a estos archivos, deberías ver:

- Archivo de fragmentos de 3 residuos (es decir, aat000_03...)
- Archivo de fragmentos de 9 residuos (es decir, aat000_09...)
- 3 archivos temporales (es decir, *.dat, *.check, *.checkpoint)
- Un archivo Fasta con tu secuencia (es decir, *.fasta)
- 3 archivos de predicción de estructura secundaria (es decir, *.jufo_ss, *.psipred, *.psipred_ss2)
- Un archivo de base de datos (es decir, *.rdb)


Solo estamos interesados en los archivos de fragmentos. Abre uno de ellos en un editor de texto para examinar su contenido. Para cada uno, se generarán 200 fragmentos, y el contenido es el siguiente:


- Column -- Significado
- 1 -- en blanco

- 2-5 -- Código PDB para el origen del fragmento

- 7 -- Identificación de cadena para el PDB de origen

- 9-13 -- Número de residuo PDB para el PDB de origen

- 15 -- Identidad del aminoácido en el PDB de origen

- 17 -- Estructura secundaria para el PDB de origen (Hélice, Bucle, Extendido/beta) 19-27 -- phi

- 28-36 -- psi

- 37-45 -- omega

- 46-54 -- Coordenada x del C-alfa para el PDB de origen (opcional)

- 55-63 -- Coordenada y del C-alfa para el PDB de origen (opcional)

- 65-73 -- Coordenada z del C-alfa para el PDB de origen (opcional)

- 74-79 -- desconocido (no utilizado)

- 80-85 -- desconocido (no utilizado)

- 86 -- Literal "P" (no utilizado)

- 87-89 -- número de posición del fragmento, numerado en la pose (no utilizado)

- 91 -- Literal "F" (no utilizado)

- 92-94 -- número del fragmento (no utilizado)

**PREGUNTA!!**
- ¿Por qué las coordenadas de posición de los fragmentos son opcionales?
- ¿Por qué no requerimos información sobre las coordenadas estructurales del resto de los átomos?


# Parte II – Ejecutar Rosetta para generar 10 modelos *ab initio*

Una vez que hayamos generado las bibliotecas de fragmentos, las emplearemos junto con Rosetta. En su núcleo, Rosetta es bastante similar a Gromacs en el sentido de que consiste en una serie de aplicaciones individuales o códigos que comparten un lenguaje y bibliotecas comunes. En este caso, utilizaremos la aplicación llamada **AbinitioRelax**. Para examinar esta aplicación en detalle, por favor, busca el nombre de la aplicación seguido de Rosetta en Google y encontrarás inmediatamente la página de documentación junto con referencias útiles. **Hazlo. En serio. ¡Rosetta es un programa extremadamente complicado y enrevesado!**


In [ ]:
#instalamos Rosetta
!pip install pyrosettacolabsetup
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()
import pyrosetta; pyrosetta.init()

Copiamos los archivos de prueba para correr el método

In [ ]:
#Descarga de Github
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/input_files/input_files_ai.zip
#Descomprimos los archivos
!unzip input_files_ai.zip

--2023-12-23 23:19:32--  https://raw.githubusercontent.com/pb3lab/ibm3202/master/input_files/input_files_ai.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1334308 (1.3M) [application/zip]
Saving to: ‘input_files_ai.zip’

input_files_ai.zip  100%[===================>]   1.27M  --.-KB/s    in 0.06s   

2023-12-23 23:19:32 (23.0 MB/s) - ‘input_files_ai.zip’ saved [1334308/1334308]

Archive:  input_files_ai.zip
  inflating: 2ppz_B.pdb              
  inflating: aat000_03_05.200_v1_3   
  inflating: aat000_09_05.200_v1_3   
  inflating: native.pdb              
  inflating: t000.dat                
  inflating: t000_.check             
  inflating: t000_.checkpoint        
 extracting: t000_.fasta             
  inflating: t000_.jufo_ss           
  inflatin

Creamos la carpeta necesaria para generar las salidas

In [ ]:
!mkdir outs

Abajo se encuentra el protocolo para correrlo desde colab

### Definition of method for folding protocol

In [ ]:
import optparse    # para opciones de ordenamiento

import pyrosetta
import pyrosetta.rosetta as rosetta

from pyrosetta import (
    init, pose_from_sequence, pose_from_file, Pose, MoveMap, create_score_function, get_fa_scorefxn,
    MonteCarlo, TrialMover, SwitchResidueTypeSetMover, PyJobDistributor,
)
from pyrosetta.rosetta import core, protocols
init(extra_options = "-scorefile_format text")

PyRosetta-4 2023 [Rosetta PyRosetta4.MinSizeRel.python310.ubuntu 2023.49+release.9891f2c59fb544fb6c9128b0af02c75b617afbf3 2023-12-05T17:48:48] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python310.ubuntu r365 2023.49+release.9891f2c59fb 9891f2c59fb544fb6c9128b0af02c75b617afbf3 http://www.pyrosetta.org 2023-12-05T17:48:48
core.init: command: PyRosetta -ex1 -ex2aro -scorefile_format text -database /usr/local/lib/python3.10/dist-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-2121938799 seed_offset=0 real_seed=-2121938799
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=-2121938799 RG_type=mt19937


In [ ]:
def sample_folding(sequence,
        long_frag_filename, long_frag_length,
        short_frag_filename, short_frag_length,
        kT = 3.0, long_inserts = 1, short_inserts = 3, cycles = 40,
        jobs = 1, job_output = 'fold_output'):
    """

    """
 # 1. crear una pose a partir de la secuencia deseada (fullatom)
# el método pose_from_sequence produce una conformación proteica completa IDEALIZADA
# de la secuencia de entrada, el ResidueTypeSet (segundo argumento abajo) puede variarse,
# y este método admite una química no proteogénica (aunque sigue siendo un Residuo de Rosetta).
# Sin embargo, esta sintaxis es más compleja y no es robusta ante errores del usuario,
# y no se presenta aquí. Pequeñas diferencias en las longitudes de enlace y ángulos de enlace
# CAMBIARÁN los resultados. Si deseas una conformación inicial alternativa, altera los pasos
# 1 y 2 según tus preferencias.
    pose = pose_from_sequence(sequence, 'fa_standard')

# 2. linearizar la pose configurando las torsiones de la cadena principal a valores grandes
# el método make_pose_from_sequence no crea el objeto PDBInfo del nuevo pose,
# por lo que se hace aquí, sin esto, se produce un error más adelante

    pose.pdb_info(rosetta.core.pose.PDBInfo( pose.total_residue() ))
    for i in range(1, pose.total_residue() + 1):
        pose.set_omega(i, 180)
        pose.set_phi(i, -150)    # reasonablemente derecho
        pose.set_psi(i, 150)
#### si deseas ver las puntuaciones de los decoys, el PDBInfo necesita estas líneas
# pose.pdb_info().chain(i, 'A')    # necesario para colorear por puntuación
# pose.pdb_info().number(i, i)    # para numeración PDB
####

    # 3. crear una copia de referencia (fullatom) del pose
    test_pose = Pose()
    test_pose.assign( pose )
    test_pose.pdb_info().name('linearized pose')

    # 4. crear Movers de conversión entre centroides y fullatom
    to_centroid = SwitchResidueTypeSetMover('centroid')
    # Los objetos Residue centroides, de aminoácidos, tienen todos sus átomos de cadena lateral
#    reemplazados por un único "átomo" representativo para acelerar los cálculos

    to_fullatom = SwitchResidueTypeSetMover('fa_standard')

    # 5. Convierte las poses a centroides
    to_centroid.apply(pose)
    to_centroid.apply(test_pose)

    # 6. crea el MoveMap, con todas las torsiones del esqueleto libre
    movemap = MoveMap()
    movemap.set_bb(True)
    # minimizar los ángulos chi del centroide (los átomos centrales de la cadena lateral) es
#    casi siempre INÚTIL ya que esta compresión se realiza para velocidad,
#    no para precisión y generalmente ocurren choques al convertir a fullatom

# 7. configurar los ClassicFragmentMovers
# para el archivo de fragmentos largos
# este "try--except" se utiliza para detectar archivos de fragmentos incorrectos

    try:
        fragset_long = core.fragment.ConstantLengthFragSet( long_frag_length , long_frag_filename )
        # el ConstantLengthFragSet está sobrecargado, este mismo
#    ConstantLengthFragSet se puede obtener con una sintaxis diferente
# para obtener fragmentos personalizados, consulte Generación de archivos de fragmentos a continuación

    except:
        raise IOError('Make sure long_frag_length matches the fragments in\n\
            long_frag_file and that long_frag_file is valid')
    long_frag_mover = protocols.simple_moves.ClassicFragmentMover(fragset_long, movemap)
    # y para el archivo de fragmentos cortos
# este "try--except" se utiliza para capturar archivos de fragmentos incorrectos
    try:
        fragset_short = core.fragment.ConstantLengthFragSet(short_frag_length, short_frag_filename)
    except:
        raise IOError('Make sure short_frag_length matches the fragments in\n\
            short_frag_file and that short_frag_file is valid')
    short_frag_mover = protocols.simple_moves.ClassicFragmentMover(fragset_short, movemap)

    # 8. setup RepeatMovers for the ClassicFragmentMovers
    insert_long_frag = protocols.moves.RepeatMover(long_frag_mover, long_inserts)
    insert_short_frag = protocols.moves.RepeatMover(short_frag_mover, short_inserts)

   # 9. crea un PyMOL_Observer para exportar estructuras a PyMOL (opcional)
    # el objeto PyMOL_Observer posee un PyMOLMover y monitorea objetos pose para
    #    cambios estructurales, cuando se detectan cambios, la nueva estructura se
    #    envía a PyMOL
    # afortunadamente, esto permite investigar protocolos completos ya que
    #    se muestran cambios intermedios, también elimina la necesidad de
    #    aplicar manualmente PyMOLMover durante un protocolo personalizado
    # desafortunadamente, esto puede dificultar la interpretación de la salida (ya que no
    #    le estás diciendo explícitamente cuándo exportar) y puede ralentizar
    #    significativamente los protocolos ya que se generan muchas estructuras (PyMOL también puede ralentizarse
    #    si se proporcionan demasiadas estructuras y una máquina rápida puede
    #    generar estructuras demasiado rápido para que PyMOL las lea, el
    #    mensaje "Buffer clean up"
    # descomenta la línea de abajo para usar PyMOL_Observer
    ##    AddPyMOLObserver(test_pose, True)

    # 10. crea ScoreFunctions
    # para poses de baja resolución, centroides, necesarias para el MonteCarlo object de TrialMover (ver abajo)
    scorefxn_low = create_score_function('score3')
    # para poses de alta resolución, fullatom, necesarias para evaluar la salida final
    #    del PyJobDistributor (ver abajo)
    scorefxn_high = get_fa_scorefxn() #  create_score_function('standard', 'score12')

   # 11. configurar un RepeatMover en un TrialMover de un SequenceMover
    # - configurar un TrialMover
    #    a. crear un SequenceMover de las inserciones de fragmentos
    #### agregar cualquier otro movimiento que desees
    folding_mover = protocols.moves.SequenceMover()
    folding_mover.add_mover(insert_long_frag)
    folding_mover.add_mover(insert_short_frag)
        #    b. crear un objeto MonteCarlo para definir éxito/fallo
    # debe restablecer el objeto MonteCarlo para cada trayectoria
    mc = MonteCarlo(test_pose, scorefxn_low, kT)
    # c. create the TrialMover
    trial = TrialMover(folding_mover, mc)
    #### para cada trayectoria, intenta "cycles" veces de aplicaciones

    # -crear el RepeatMover
    folding = protocols.moves.RepeatMover(trial, cycles)

    # 12. create a (Py)JobDistributor
    jd = PyJobDistributor(job_output, jobs, scorefxn_high)

     # 13. almacenar las evaluaciones de puntaje para la salida
    # imprimir los puntajes a medida que se producen sería difícil de leer,
    #    Rosetta produce una gran cantidad de salida detallada al ejecutarse
    scores = [0]*(jobs + 1)
    scores[0] = scorefxn_low(pose)

    # 14. perform folding by
    counter = 0    # for exporting to PyMOL
    while not jd.job_complete:
        # a. set necessary variables for the new trajectory
        # -reload the starting pose
        test_pose.assign(pose)
        # -change the pose's PDBInfo.name, for the PyMOL_Observer
        counter += 1
        test_pose.pdb_info().name(job_output + '_' + str(counter))
        # -reset the MonteCarlo object (sets lowest_score to that of test_pose)
        mc.reset(test_pose)

        #### si creas un protocolo personalizado, es posible que tengas variables adicionales
        ####    para restablecer, como kT

        #### si creas un protocolo personalizado, es probable que esta sección
        ####    cambie; muchos protocolos existen como Movers individuales o se pueden
        ####    encadenar en una secuencia (ver arriba), por lo que solo necesitas
        ####    aplicar el Mover final
        # b. aplicar el protocolo de refinamiento
        folding.apply(test_pose)

        ####
        # c. exportar la estructura del señuelo de menor puntuación para esta trayectoria
        # -recuperar la estructura del señuelo de menor puntuación
        mc.recover_low(test_pose)
        # -almacenar la puntuación final para esta trayectoria
        scores[counter] = scorefxn_low(test_pose)
        # -convertir el señuelo a fullatom
        # las conformaciones de las cadenas laterales serán las predeterminadas,
        #    normalmente, los señuelos NO se convertirían a fullatom antes de
        #    escribirlos en un archivo PDB (ya que se consideraría un gran número de trayectorias y
        #    sus puntuaciones de fullatom no son necesarias)
        # aquí se reproduce el modo fullatom para facilitar la comprensión y manipulación del resultado;
        #    PyRosetta puede cargar archivos PDB de estructuras centroides, sin embargo, debes convertirlos a fullatom para
        #    casi cualquier otra aplicación
        to_fullatom.apply(test_pose)
        # -guess what cysteines are involved in disulfide bridges
        guess_disulfides(test_pose)
        # -output the fullatom decoy structure into a PDB file
        jd.output_decoy(test_pose)
        # -export the final structure to PyMOL
        test_pose.pdb_info().name(job_output + '_' + str(counter) + '_fa')

        #### if you want to see the decoy scores, uncomment the line below
        #scorefxn_high( test_pose )

    # 15. output the score evaluations
    print( '===== Centroid Scores =====' )
    print( 'Original Score\t:\t', scores[0] )
    for i in range(1, len(scores)):    # print out the job scores
        # the "[:14].ljust(14)" is to force the text alignment
        print( (job_output + '_' + str( i ))[:14].ljust(14) +\
            '\t:\t', scores[i] )

    return scores    # for other protocols

# locates all cysteines, make bonds between them, output the bonds that lower
#    the score
def guess_disulfides(pose, cutoff = 6.0):
    """
    A quick method for probing a protein for cysteine residues close to each
        other (within  <cutoff>  )
    """
    # find all cysteine residues and consider possible disulfides
    cys = [i for i in range(1, pose.total_residue() + 1)
        if pose.residue(i).name1() == 'C']
    partners = [0]*sum( range(len(cys)) )    # all disulfides possible
    i = 0
    # create all combinations
    for first in range(len(cys[:-1])):
        for second in cys[first + 1:]:
            partners[i] = (cys[first], second)
            i += 1
    # try each disulfide, if it lowers the score, print it to screen
    print( '='*80 )
    print( 'Potential Disulfides:' )
    for pair in partners:
        # for a fullatom cysteine in PyRosetta, the 6th atom is sulfur
        separation = (pose.residue(pair[0]).xyz(6) -
            pose.residue(pair[1]).xyz(6)).norm
        if separation < cutoff:
            print( 'between (pose numbered) residue' , pair[0] , 'and' ,\
                pair[1] , '|' , separation , 'Angstrom separation' )
    print( '='*80 )
    # para manipular enlaces disulfuro, usa:
    # formación:    form_disulfide(pose.conformation(), 6, 16)
    # ruptura:     change_cys_state(6, 'CYS', pose.conformation() )
    #               change_cys_state(16, 'CYS', pose.conformation() )


## Aquí definimos las variables de entrada

- La longitud de `long_frag_length` **DEBE** coincidir con `long_frag_filename` (9 en este ejemplo).
- La longitud de `short_frag_length` **DEBE** coincidir con `short_frag_filename` (3 aquí).
- **100 trayectorias** son pocas; muestrear conformaciones de proteínas requiere muchos intentos, típicamente cientos (800-1000) de trayectorias se intentan.
- El parámetro de **"temperatura"** kT se establece en 1.0, un valor neutral. Valores más grandes de kT aumentan la diversidad del muestreo (permiten escapar fácilmente de mínimos locales) pero requieren compensación con más trayectorias. Valores más pequeños de kT disminuyen la probabilidad de muestrear espacios "inútiles" pero no pueden escapar fácilmente de mínimos locales (el valor predeterminado de kT = 3.0).
- No hay valores comunes para `long_inserts`, `short_inserts` o `cycles`. Más inserciones indican un mayor muestreo con fragmentos, mientras que más ciclos producen un muestreo más general (aunque en este caso solo se realiza la inserción de fragmentos). Sin embargo, solo se aplica una selección (Criterio de Metrópolis) por ciclo, por lo que es probable que se rechace una estructura propuesta con demasiados fragmentos insertados, pero más ensayos aumentan notablemente el tiempo de cálculo. Aquí, como es típico en Rosetta, se propone un cambio bastante conservador (1 fragmento de 9-mer y 3 fragmentos de 3-mers) por ciclo con muchos ciclos.


In [ ]:
# el usuario puede ingresar un archivo PDB, un archivo fasta o una secuencia directamente
# opción de archivo PDB
pose = Pose()
# Opción de archivo Fasta
fasta_filename = './t000_.fasta'
if fasta_filename:    # por defecto está desactivada, cadena vacía
    f = open(fasta_filename, 'r')    # abre el archivo
    sequence = f.readlines()    # lee el texto
    f.close()    # ciérralo
    # eliminando el "\n" final y cualquier línea de encabezado
    sequence = [line.strip() for line in sequence if not '>' in line]
    sequence = ''.join(sequence)    # combina en una sola secuencia
elif options.sequence:
    sequence = options.sequence
#else:
 #   pose_from_file(pose, pdb_filename)
 #   sequence = pose.sequence()
# Comprueba la secuencia en un archivo fasta, luego directamente y finalmente desde un archivo PDB. Si no se proporciona un archivo PDB, cargará el predeterminado.

# opciones de archivos de fragmentos
long_frag_filename = './aat000_09_05.200_v1_3'
long_frag_length = 9
short_frag_filename = './aat000_03_05.200_v1_3'
short_frag_length = 3
# opciones del protocolo de plegamiento, ver arriba
kT = 3.0
long_inserts = 1
short_inserts = 3
cycles = 300
# opciones de PyJobDistributor
jobs = 10
job_output = './outs/fold_output'                  # Prefijo

## Y finalmente llamamos la función de plegamiento

In [ ]:
sample_folding(sequence,
    long_frag_filename, long_frag_length,
    short_frag_filename, short_frag_length,
    kT, long_inserts, short_inserts, cycles,
    jobs, job_output)

NameError: ignored

# Parte III – Analizar los resultados

Como resultado de ejecutar Rosetta, ocurrirán dos cosas:
1. Se generarán todos los archivos PDB de las 10 estructuras predichas para la cabeza de villin.
2. También se generará continuamente un archivo de puntuación de energía y estadísticas de la salida, llamado score.fsc.


In [ ]:
!sudo apt install jq

In [ ]:
# Dado que obtenemos un archivo de puntuación Json, usamos el analizador json jq de bash para filtrar el nombre del señuelo y total_score
# con el fin de obtener el modelo de menor energía
!jq '.decoy, .total_score' /content/outs/fold_output.fasc

Ahora es tu turno de tomar la estructura PDB de 2PPZ (native.pdb) y alinearla con la estructura del modelo predicho con la energía más baja utilizando py3Dmol y BioPython.

In [ ]:
!pip install biopython
#importamos bio
from Bio.PDB import *
parser = PDBParser()

In [ ]:
# El siguiente código fue creado por Anders Steen Christensen de la Universidad de Basilea y está disponible en [este enlace](https://gist.github.com/andersx/6354971).

import Bio.PDB

# Selecciona los números de residuos que deseas alinear y colócalos en una lista
start_id = 1
end_id   = 35
atoms_to_be_aligned = range(start_id, end_id + 1)

# Inicia el analizador
pdb_parser = Bio.PDB.PDBParser(QUIET=True)

# Obtiene las estructuras
ref_structure = pdb_parser.get_structure("reference", "/content/native.pdb")
sample_structure = pdb_parser.get_structure("sample", "PathToYourBestDecoy!!.pdb")

# Utiliza el primer modelo en los archivos PDB para el alineamiento
# Cambia el número 0 si deseas alinear con otra estructura
ref_model    = ref_structure[0]
sample_model = sample_structure[0]

# Crea listas de átomos (en las estructuras) que deseas alinear.
# En este caso, se utilizan átomos CA cuyos índices están en el rango especificado
ref_atoms = []
sample_atoms = []

# Itera sobre todas las cadenas en el modelo para encontrar todos los residuos
for ref_chain in ref_model:
  # Itera sobre todos los residuos en cada modelo para encontrar los átomos apropiados
  for ref_res in ref_chain:
    # Verifica si el número de residuo (.get_id()) está en la lista
    if ref_res.get_id()[1] in atoms_to_be_aligned:
      # Agrega el átomo CA a la lista
      ref_atoms.append(ref_res['CA'])

# Haz lo mismo para la estructura de muestra
for sample_chain in sample_model:
  for sample_res in sample_chain:
    if sample_res.get_id()[1] in atoms_to_be_aligned:
      sample_atoms.append(sample_res['CA'])

# Ahora iniciamos el superponedor:
super_imposer = Bio.PDB.Superimposer()
super_imposer.set_atoms(ref_atoms, sample_atoms)
super_imposer.apply(sample_model.get_atoms())

# Imprime el RMSD:
print('El RMSD calculado es:')
print(super_imposer.rms)

# Guarda la versión alineada de una de las cadenas de 6ANE
io = Bio.PDB.PDBIO()
io.set_structure(sample_structure)
io.save("Your_best_model_aligned.pdb")

In [ ]:
#instalamos py3Dmol
!pip install py3Dmol

In [ ]:
#importamos los módulos
import py3Dmol
view=py3Dmol.view()
view.addModel(open('/content/Your_best_model_aligned.pdb', 'r').read(),'pdb')
view.addModel(open('/content/native.pdb', 'r').read(),'pdb')
view.zoomTo()
view.setBackgroundColor('white')
view.setStyle({'chain':'A'},{'cartoon': {'color':'yellow'}})
view.setStyle({'chain':'B'},{'cartoon': {'color':'green'}})
view.show()

# Apéndice A. Profundización

## Un ejemplo real

Todas las variables y parámetros predeterminados utilizados anteriormente son específicos del ejemplo con "test_in.pdb", que se supone que es simple, directo y rápido. Aquí tienes un ejemplo más práctico:

La feromona ER-23 es una proteína pequeña (51 aminoácidos) con 5 puentes disulfuro. La formación de estos enlaces puede afectar significativamente el plegamiento. Supongamos que te interesa la importancia de estos enlaces disulfuro como restricciones de plegamiento y deseas predecir la proteína sin ellos utilizando PyRosetta.

"""
1. Obtén la secuencia de proteínas de RCSB PDB 1HA8 (instrucciones arriba)
        (para el ejemplo aquí, descarga el archivo PDB y asegúrate de que esté limpio)
2. Crea dos archivos de fragmentos utilizando "Generate Fragment Files"
        (instrucciones arriba)
        - de 9-meros (los fragmentos largos)
        - de 3-meros (los fragmentos cortos)
3. Crea un directorio que contenga:
        - el archivo PDB para 1HA8 (limpio de HETATMs y aguas)
            llamémoslo "1HA8.clean.pdb" aquí
        - el archivo de fragmentos de 9-meros para 1HA8
            llamémoslo "1HA8.frag9" aquí
        - el archivo de fragmentos de 3-meros para 1HA8
            llamémoslo "1HA8.frag3" aquí
        - este script de ejemplo (técnicamente no es necesario, pero de lo contrario,
            los comandos en el paso 4 cambiarían ya que folding.py no estaría aquí)
4. Ejecuta el script desde la línea de comandos con los argumentos apropiados:

>   python folding.py --pdb_filename 1HA8.clean.pdb --long_frag_filename 1HA8.frag9 --long_frag_length 9 --short_frag_filename 1HA8.frag3 --short_frag_length 3 --jobs 100 --job_output 1HA8_folding_output --kT 1.0 --long_inserts 1 --short_inserts 3 --cycles 200 --disulfides 1

        - La longitud de long_frag_length DEBE coincidir con long_frag_filename (9 en este ejemplo).
- La longitud de short_frag_length DEBE coincidir con short_frag_filename (3 aquí).
- 100 trayectorias son pocas, el muestreo de conformaciones de proteínas requiere muchos
  ensayos, típicamente se intentan cientos (800-1000) de trayectorias.
- El parámetro "temperature" (kT) se establece en 1.0, un valor neutral. Un kT más grande
  aumenta la diversidad de muestreo (escapa fácilmente de los mínimos locales) pero requiere
  compensación con más trayectorias. Un kT más pequeño disminuye la posibilidad de muestreo
  de espacio "inútil" pero no puede escapar fácilmente de los mínimos locales (el kT predeterminado es 3.0).
- No hay valores comunes para long_inserts, short_inserts o cycles.
  Más inserciones indican un mayor muestreo con fragmentos, mientras que
  más ciclos producen más muestreo en general (aunque en este caso solo se realiza la inserción de fragmentos).
  Sin embargo, solo se aplica una selección (Criterios de Metropolis) por ciclo, por lo que es probable
  que se rechace una estructura propuesta con demasiados fragmentos insertados, pero
  se aumenta el tiempo de computación notablemente. Aquí, como es típico en Rosetta,
  se propone un cambio bastante conservador (1 9-mer y 3 3-mers) por ciclo con muchos ciclos.


5. Espera la salida; esto llevará un tiempo (realizando 100 trayectorias
   de modelado de bucles que involucran 1*3*200 movimientos totales por trayectoria).
6. Analiza los resultados (ver la sección INTERPRETACIÓN DE RESULTADOS arriba).

Nota: esto NO está destinado a ser utilizado para un plegamiento centrado realista,
   simplemente proporciona un "esqueleto" para el código en PyRosetta. Puede ser útil
   para investigaciones preliminares, pero los mejores protocolos son algo
   específicos para la proteína; no hay un método universal actual para el plegamiento.
   Como se mencionó anteriormente, este script solo aborda el aspecto de baja resolución
   del plegamiento de Rosetta y debe ir acompañado de un paso de alta resolución para
   obtener resultados completos. En general, un protocolo similar al presentado aquí con
   un muestreo más drástico y un mayor número de ensayos debería ser suficiente para
   encontrar plegamientos realistas (después del refinamiento).


"""




## Cambiando la exploración de plegamiento
"""
Rosetta es una herramienta de predicción de estructuras y todas las predicciones dependen de las
técnicas (movimientos) utilizadas para generar las estructuras propuestas. NO EXISTE UNA SOLUCIÓN
GENERAL para predecir el plegamiento de una proteína desde cero, y la inserción de fragmentos es
una manera efectiva de muestrear el espacio relevante. Dado que los fragmentos dependen de la
secuencia y la estructura secundaria (predicción), los plegamientos resultantes son
bioquímicamente factibles y están en conformidad con nuestras observaciones previas sobre la
estructura de las proteínas (el PDB).

Por velocidad, el plegamiento de Rosetta (y el algoritmo de plegamiento presentado aquí) se realiza
en modo de baja resolución (centroides). Este método de plegamiento busca abstractamente estructuras
que cumplan con ciertas restricciones de tamaño (es decir, el término de puntuación rg), pero que no
choquen de manera irrealista (es decir, el término de puntuación vdw) y que se logren fácilmente
(es decir, la inserción de fragmentos, un algoritmo rápido). Un muestreo suficiente y el refinamiento
posterior convertirán estos plegamientos generales (la salida de este script) en estimaciones
razonables de la estructura nativa de la proteína.

Existen muchas otras técnicas y métodos de muestreo disponibles. PyRosetta fue diseñado para facilitar
la investigación novedosa de estas ideas. Se recomienda la literatura que aborda temas como la predicción
de la estructura de proteínas, procesos de cadenas de Markov de Monte Carlo, protocolos de Rosetta y
algoritmos de recocido simulado. Al desarrollar protocolos personalizados, DEBES entender los Movers
(dispositivos) disponibles en PyRosetta (consulta los otros scripts de muestra, específicamente
refinement.py). Desafortunadamente, no puedes construir nuevas clases de Movers en PyRosetta.
Afortunadamente, es fácil implementar código similar por tu cuenta una vez que entiendes cómo se
realizan los movimientos y la selección.

Por favor, prueba métodos de muestreo alternativos para entender mejor cómo funcionan estos algoritmos
y encontrar qué movimientos se adaptan mejor a tu problema.


"""




## Cambiando la puntuación del plegamiento
"""
El "force-field" de Rosetta es una función de sesgo (ver algoritmos de MCMC) utilizada para predecir
estructuras de proteínas basada en lo observado en otras estructuras de proteínas. Estos métodos de
puntuación proporcionan las restricciones necesarias para sesgar la predicción de estructuras, pero
incorporan un GRAN número de parámetros ajustados finamente. No hay una manera fácil de optimizar
todas las constantes relevantes y el rendimiento se ha mejorado al equilibrar velocidad y precisión.

Existen muchas otras métricas relevantes para las estructuras de proteínas y discriminar entre decoys
(predicciones incorrectas) y proteínas reales es una tarea en curso. Por favor, experimenta con
diferentes ScoreTypes en PyRosetta, sus pesos y la combinación de términos. Desafortunadamente, no
puedes construir nuevos ScoreTypes (métodos de puntuación) en PyRosetta. Afortunadamente, es fácil
implementar código similar por tu cuenta una vez que entiendes cómo la puntuación sesga las simulaciones
de MCMC.

Por favor, prueba funciones de puntuación alternativas o métodos de selección únicos para entender mejor
qué términos de puntuación contribuyen al rendimiento y encontrar qué puntuación se adapta mejor a tu
problema.


"""